In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Loading modules

In [2]:
import mbuild as mb
import foyer
from foyer import Forcefield

import mbuild.formats.charmm_writer as mf_charmm

import mosdef_cp2k_writer


import parmed as pmd
from constrainmol import ConstrainedMolecule
import unyt as u

from mosdef_cp2kmcpy.mc import MC
import setter
warnings.filterwarnings("ignore", category=DeprecationWarning)
 

## The forcefield file

In [3]:
!cat water_ff.xml

<ForceField>
 <AtomTypes>
  <Type name="o_fake_spce" class="ow" element="O" mass="15.999" def="[O;X2](H)(H)" desc="fake water oxygen" doi="10.1063/1.5025184"/>
  <Type name="h_fake_spce" class="hw" element="H" mass="1.008"  def="H[O;X2](H)" desc="fake water hydrogen" doi="10.1063/1.5025184"/>
 </AtomTypes>
 <HarmonicBondForce>
  <Bond class1="ow" class2="hw" length="0.1" k="502416.0"/>
  </HarmonicBondForce>
 <HarmonicAngleForce>
  <Angle class1="hw" class2="ow" class3="hw" angle="1.91063324" k="383.0"/>
 </HarmonicAngleForce>
 <NonbondedForce coulomb14scale="0.5" lj14scale="0.5">
  <Atom type="o_fake_spce" charge="-0.4238" sigma="0.31656" epsilon="0.162525"/>
  <Atom type="h_fake_spce" charge="0.2119"  sigma="0.0"    epsilon="0.0"/>
  </NonbondedForce>
</ForceField>


## Defining the molecule



In [4]:

water_res_name = 'H20'
FF_file_water = 'water_ff.xml'
water_FF = Forcefield(forcefield_files=FF_file_water)
water_mb = mb.load('O', smiles=True)



/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/foyer/forcefield.py:448: UserWarning: No force field version number found in force field XML file.
  warnings.warn(
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/foyer/forcefield.py:460: UserWarning: No force field name found in force field XML file.
  warnings.warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:687: UserWarning: No unitcell detected for pybel.Molecule O	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))


In [5]:
water_mb.name = water_res_name

water_typed=water_FF.apply(water_mb)





In [17]:
constrain_mol = ConstrainedMolecule(water_typed)
constrain_mol.solve()
water_optimized_mb=mb.clone(water_mb)
water_optimized_mb.xyz=constrain_mol.xyz/10
print(water_optimized_mb.xyz)

[[ 0.09289464  0.00637479 -0.00348477]
 [ 0.19268044  0.00471668 -0.00981287]
 [ 0.06580068 -0.01802386 -0.09660095]]


In [18]:
water_optimized_mb.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Creating .psf (Topology) and .inp (CHARMM potential file) using mosdef_charmm_writer


In [19]:
#random box to be used in mosdef_charmm_writer
water_box_bias  = mb.fill_box(compound=[water_optimized_mb ],
                             n_compounds=[1] ,
                            box=[1.0, 1.0, 1.0] )

### mosdef_charmm_writer

In [20]:
FF_Dict = {water_optimized_mb.name:FF_file_water }

residues_List = [water_optimized_mb.name ]




water_box_bias.save("water_bias_coord.xyz",overwrite=True)

print('Running: GOMC FF file, and the psf and pdb files for the biasing potential file')
mf_charmm.charmm_psf_psb_FF(water_box_bias ,
                            'water_bias',
                            FF_filename ="water_charmm_bias" ,
                            forcefield_selection = FF_Dict,
                            residues= residues_List ,
                            bead_to_atom_name_dict = {},
                            fix_residue = None,
                            reorder_res_in_pdb_psf = False
                            )

! rm *.pdb *.xyz

Running: GOMC FF file, and the psf and pdb files for the biasing potential file
write_gomcdata: forcefield_selection = {'H20': 'water_ff.xml'}, residues = ['H20']
******************************

GOMC FF writing each residues FF as a group for structure_0
forcefield type from compound = {'H20': 'water_ff.xml'}
coulomb14scale from compound = {'H20': 0.5}
lj14scale from compound = {'H20': 0.5}
unique_types = ['h_fake_spce_H20', 'o_fake_spce_H20']
No urey bradley terms detected, will use angle_style harmonic
******************************

writing the GOMC force field file 
NBFIX_Mixing not used or no mixing used for the non-bonded potentials out
forcefield_dict = {1: 'o_fake_spce_H20', 0: 'h_fake_spce_H20'}
******************************

write_charmm_psf file is running
write_charmm_psf: forcefield_selection = {'H20': 'water_ff.xml'}, residues = ['H20']
******************************

No urey bradley terms detected
******************************

write_charmm_pdb file is running
write_ch

### Now we have the .psf (topology) file and .inp (charmm potential) file

## Creating CP2K MC simulation

In [21]:
molecule_list=[water_optimized_mb]
box=mb.box.Box(lengths=[1,1,1])
#Comment why pressure is needed
q=MC(molecule_list=molecule_list,n_box=2,n_molecules_each_box=[[49],[1]], box_list=[box,box],cutoff=200,functional='PBE',
     basis_set={'N':'DZVP-MOLOPT-GTH','H':'DZVP-MOLOPT-GTH' ,'O':'DZVP-MOLOPT-GTH'}, periodicity=['XYZ']*2,ensemble='GEMC_NVT',seed=1,project_name='NH3_NVT_GEMC',restart='FALSE',pressure=1*u.bar)

In [22]:
q.mc_initialization()

scf_tolerance not specified, set as 1e-6
basis_set_filename not defined, set as BASIS_MOLOPT
potential_filename not specified, set as GTH_POTENTIALS
n_steps not specified, set as 1000
n_ff_moves not specified, set as 8
nswapmoves not specified, set as 640, will be ignore if n_box<2
output trajectory format set as XYZ
input_filename not specified, set as ['GEMC_NVT_box1.inp', 'GEMC_NVT_box2.inp']
output_filename not specified, set as NH3_NVT_GEMC_mc_output.out
temperature not defined, set as 298 K
You can change default settings in setter.mc_files


In [23]:

q.topology_filename=['water_bias.psf']

# move_probabilities=[pmavbmc,pmcltrans,pmhmc,pmswap,pmtraion,pmtrans,pmvolume]
#volume moves = PMVOLUME
# swap moves = PMSWAP - PMVOLUME
# AVBMC moves = PMAVBMC - PMSWAP - PMVOLUME
# an “inner” move = 1.0 - (PMAVBMC + PMSWAP + PMVOLUME)
#conformational changes = “inner” move percentage × PMTRAION
# molecular translation = “inner” move percentage × (PMTRANS - PMTRAION)
# molecular rotation = “inner” move percentage × (1.0 - PMTRANS - PMTRAION)

q.move_probabilities=[0,0.0,0,0.4,0.5,0.75,0.05]



# mol_probabilities=[[PMAVBMC_MOL,PMSWAP_MOL , PMTRAION_MOL, PMTRANS_MOL,PMROT_MOL],[]]
q.mol_probabilities=[[[1],[1],[1],[1],[1]],[[1],[1],[1],[1],[1]]]



#avbmc probabilities=[[AVBMC_ATOM,AVBMC_RMIN,AVBMC_RMAX,PBIAS],[]]
q.avbmc_probabilities=[[[1],[1],[1],[1]],[[1],[1],[1],[1]]]

q.charmm_potential_file='water_charmm_bias.inp'

In [24]:
setter.mc_files(q)

MC initial structure saved as ['NH3_NVT_GEMC_box1_initial.xyz', 'NH3_NVT_GEMC_box2_initial.xyz']
MC input file saved as GEMC_NVT_box1.inp
MC input file saved as GEMC_NVT_box2.inp


In [25]:
!ls

GEMC_NVT_box1.inp		       h2_ff.xml
GEMC_NVT_box2.inp		       hydrogen_bias.psf
NH3_NVT_GEMC_SCF_DM_SPIN_1_RESTART.dm  hydrogen_charmm_bias.inp
NH3_NVT_GEMC_box1_initial.xyz	       hydrogen_gemc.ipynb
NH3_NVT_GEMC_box2_initial.xyz	       md-changeLog.out
__pycache__			       setter.py
bias_temp.dat			       water_bias.psf
bias_template.inp		       water_charmm_bias.inp
changeLog.out			       water_ff.xml
combined_GEMC.ipynb		       water_gemc.ipynb


In [35]:
!cp2k.popt -i GEMC_NVT_box1.inp -i GEMC_NVT_box2.inp

 DBCSR| Multiplication driver                                               BLAS
 DBCSR| Multrec recursion limit                                              512
 DBCSR| Multiplication stack size                                           1000
 DBCSR| Maximum elements for images                                    UNLIMITED
 DBCSR| Multiplicative factor virtual images                                   1
 DBCSR| Multiplication size stacks                                             3
 DBCSR| Number of 3D layers                                               SINGLE
 DBCSR| Use MPI memory allocation                                              T
 DBCSR| Use RMA algorithm                                                      F
 DBCSR| Use Communication thread                                               T
 DBCSR| Communication thread load                                             87


  **** **** ******  **  PROGRAM STARTED AT               2022-09-27 12:10:49.203
 ***** ** ***  *** **   PR


 *******************************************************************************
 *******************************************************************************
 **                                                                           **
 **     #####                         ##              ##                      **
 **    ##   ##            ##          ##              ##                      **
 **   ##     ##                       ##            ######                    **
 **   ##     ##  ##   ##  ##   #####  ##  ##   ####   ##    #####    #####    **
 **   ##     ##  ##   ##  ##  ##      ## ##   ##      ##   ##   ##  ##   ##   **
 **   ##  ## ##  ##   ##  ##  ##      ####     ###    ##   ######   ######    **
 **    ##  ###   ##   ##  ##  ##      ## ##      ##   ##   ##       ##        **
 **     #######   #####   ##   #####  ##  ##  ####    ##    #####   ##        **
 **           ##                                                    ##        **
 **                        


 Number of electrons:                                                          8
 Number of occupied orbitals:                                                  4
 Number of molecular orbitals:                                                 4

 Number of orbital functions:                                                 23
 Number of independent orbital functions:                                     23

 Extrapolation method: initial_guess

 *** WARNING in qs_initial_guess.F:263 :: User requested to restart the    ***
 *** wavefunction from the file named: NH3_NVT_GEMC-RESTART.wfn. This file ***
 *** does not exist. Please check the existence of the file or change      ***
 *** properly the value of the keyword WFN_RESTART_FILE_NAME. Calculation  ***
 *** continues using ATOMIC GUESS.                                         ***



 SCF WAVEFUNCTION OPTIMIZATION

  ----------------------------------- OT ---------------------------------------
  Minimizer      : DIIS                : di

^C
